In [ ]:
# movielens 100K

#SVD 알아오기
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pathlib import Path

path = Path('../../../datafile/Recsys/ml-100k')
train = pd.read_csv(path /'ub.base',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})
test = pd.read_csv(path /'ub.test',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})

R_train = pd.pivot_table(train,values='rating',index='user',columns='item')

In [2]:
import matfac
mf_model = matfac.MatrixFactorization(k=10,lr=0.007,reg_param=0.1,epochs=5)
R_train = R_train.fillna(0)
 
print('Start Model Training')
P,Q,bu,bi = mf_model.fit(R_train)
print('\nModel Training Success')


Start Model Training
Epoch : 0 , Loss : 220.178306 , Rooted Loss: 14.84

Model Training Success


In [3]:
#Prediction
prediction,test_filtered = mf_model.predict(test,exclude_unknowns=False)    

c:\Users\hanan\.vscode\DS\vscode\Study\RecSys\matrixfactorization\matfac.py:96: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bias_u_product = int(bu_df.loc[val[0]])
c:\Users\hanan\.vscode\DS\vscode\Study\RecSys\matrixfactorization\matfac.py:92: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bias_i_product = int(bi_df.loc[val[1]])


In [6]:
prediction.sort_values(by='user').loc[prediction['user']==102]

,user,item,rating
1018,102,732,3.064346
1019,102,1025,3.064346
1016,102,559,3.064346
1017,102,576,3.064346
1011,102,264,3.064346
1010,102,118,3.064346
1014,102,391,3.064346
1015,102,432,3.064346
1013,102,313,3.064346
1012,102,300,3.064346


In [5]:
test_filtered.sort_values(by='user').loc[prediction['user']==101]

,user,item,rating
1008,101,1051,2.0
1009,101,1057,2.0
1001,101,109,2.0
1000,101,24,4.0
1006,101,928,2.0
1007,101,1034,2.0
1005,101,840,3.0
1004,101,815,3.0
1003,101,121,4.0
1002,101,118,3.0


In [19]:
P[100,:]

array([-0.30367985, -0.20961275,  0.00200454,  0.20671805, -0.34120335,
       -0.21009185, -0.25848443, -0.32632343, -0.01076573, -0.37459926])

In [24]:
mean = R_train[R_train>0].mean().mean()
np.dot(P[100,:],Q[23,:])+bu[100]+bi[22]+mean

np.float64(3.288727745291278)

In [27]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9
0,0.183824,-0.392086,0.089765,-0.262281,-0.028810,0.025900,-0.021809,-0.130175,-0.036907,-0.152414
1,0.291894,-0.297438,-0.063916,-0.074590,-0.301620,-0.086054,-0.007579,0.106535,-0.066924,-0.098487
2,0.136816,0.446616,-0.353256,-0.032251,-0.630796,0.647178,-0.253721,-0.139486,-0.307504,-0.033549
3,0.077607,0.189323,-0.003159,-0.253901,-0.025583,0.065216,-0.396184,-0.283012,0.106474,-0.087222
4,-0.152011,-0.354846,0.153827,0.391834,-0.116958,-0.044822,0.046543,0.217901,-0.214913,0.100239
...,...,...,...,...,...,...,...,...,...,...
1670,0.519072,0.126512,0.373066,0.280365,-0.033232,0.149968,0.524701,0.419689,0.119111,0.768029
1671,0.171733,0.158618,0.494881,0.524029,0.190080,0.307997,0.234535,0.349172,0.527500,0.065811
1672,0.571361,-0.086608,0.121815,0.260735,0.199646,0.515910,0.352522,0.050565,0.080549,0.507698
1673,0.540240,0.052326,0.260526,0.161094,0.042836,0.106486,0.465406,0.151775,0.578712,0.090735


In [5]:
prediction.sort_values(by='user').loc[prediction['user']==101]

,user,item,rating
1008,101,1051,3.064346
1009,101,1057,3.064346
1001,101,109,3.064346
1000,101,24,3.064346
1006,101,928,3.064346
1007,101,1034,3.064346
1005,101,840,3.064346
1004,101,815,3.064346
1003,101,121,3.064346
1002,101,118,3.064346


In [66]:
P_df = pd.DataFrame(P,index=R_train.index)
Q_df = pd.DataFrame(Q,index=R_train.columns)

In [ ]:
uilist_train = list(zip(train['user'],train['item']))
test_filtered = test[test['user'].isin(uilist_train[0]) | test['item'].isin(uilist_train[1])]
test_filtered
uilist_test = list(zip(test_filtered['user'],test_filtered['item']))

In [ ]:
uilist_test = list(zip(test['user'],test['item']))
pred = []
for val in uilist_test:
    pred.append(np.dot(P_df.loc[val[0]],Q_df.loc[val[1]].T))
test['prediction'] = pred  
prediction = test.copy()
prediction['rating'] = pred

In [4]:
from sklearn.metrics import root_mean_squared_error
final_rmse = root_mean_squared_error(prediction['rating'].values,test_filtered['rating'].values)
print(f'RMSE: {final_rmse:.4f}')

RMSE: 4.8693


In [36]:
c = test.loc[(test['user']==1) & (test['rating']>=0.3)].sort_values('rating',ascending=False)[:10]
d = prediction.loc[(test['user']==1) & (test['rating']>=0.3)].sort_values('rating',ascending=False)[:10]
hit = set(c['item']).intersection(set(d['item']))
len(hit) / 10

0.0

In [27]:
a= test.loc[test['user']==1].sort_values('rating',ascending=False)[:30]
b= test.loc[test['user']==1].sort_values('prediction',ascending=False)[:30]
hit = set(a['item']).intersection(set(b['item']))
hit

{12, 60, 100}

In [43]:
test.sort_values(['user','rating']).reset_index(drop=True)

,user,item,rating,prediction
0,1,74,1.0,-4.636218
1,1,78,1.0,-4.293500
2,1,103,1.0,3.207766
3,1,104,1.0,6.743181
4,1,112,1.0,-3.364657
...,...,...,...,...
19995,458,648,4.0,0.874911
19996,458,1101,4.0,0.606245
19997,459,934,3.0,5.956687
19998,460,10,3.0,-2.650224


In [5]:
uilist_test = list(zip(test['user'],test['item']))
pred_list = []
for ind in uilist_test:
    pred_list.append(R_train[ind[0],ind[1]])
pred_list

KeyError: (877, 381)

In [ ]:
user_item_mat = pd.pivot_table(data=raw_data,values='rating',columns='user',index='item')
user_item_mat

user,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import numpy as np

# 분해할 행렬 A 생성
A = np.array([
    [3, 1, 1],
    [-1, 3, 1]
])

print("--- 원본 행렬 A ---")
print(A)

U, s, Vt = np.linalg.svd(A)

print("\n--- 분해된 행렬들 ---")
print("U의 모양:", U.shape)
print("s의 모양:", s.shape) # 1차원 배열임에 주의!
print("Vt의 모양:", Vt.shape)

# s를 대각 행렬 Σ로 재구성
# 원본 A의 크기에 맞게 0을 채워줌
Sigma = np.zeros(A.shape)
Sigma[:len(s), :len(s)] = np.diag(s)

k = 2
U_k = U[:,:k]
Sigma_k = Sigma[:k,:k]
Vt_k = Vt[:k,:]

print("\n--- 재구성된 대각 행렬 Σ ---")
print(Sigma)

# U, Σ, Vt를 다시 곱해서 A를 복원
A_restored = U @ Sigma @ Vt  # @는 행렬 곱셈 연산자
A_k_restored = U_k @ Sigma_k @ Vt_k

print("\n--- 복원된 행렬 A,A_k ---")
print('Truncated SVD: ',A_k_restored.round(3))
print('Full-SVD : ',A_restored.round(3)) # 부동소수점 오차를 감안해 반올림

--- 원본 행렬 A ---
[[ 3  1  1]
 [-1  3  1]]

--- 분해된 행렬들 ---
U의 모양: (2, 2)
s의 모양: (2,)
Vt의 모양: (3, 3)

--- 재구성된 대각 행렬 Σ ---
[[3.46410162 0.         0.        ]
 [0.         3.16227766 0.        ]]

--- 복원된 행렬 A,A_k ---
Truncated SVD:  [[ 3.  1.  1.]
 [-1.  3.  1.]]
Full-SVD :  [[ 3.  1.  1.]
 [-1.  3.  1.]]
